케라스(Keras) - 그 간결함에 빠지다.

케라스는 파이썬으로 구현된 쉽고 간결한 딥러닝 라이브러리입니다. 딥러닝 비전문가라도 각자 분야에서 손쉽게 딥러닝 모델을 개발하고 활용할 수 있도록 케라스는 직관적인 API를 제공하고 있습니다. 내부적으로는 텐서플로우(TensorFlow), 티아노(Theano), CNTK 등의 딥러닝 전용 엔진이 구동되지만 케라스 사용자는 복잡한  내부 엔진을 알 필요는 없습니다. 직관적인 API로 쉽게 다층퍼셉트론 모델, 컨볼루션 신경망 모델, 순환 신경망 모델 또는 이를 조합한 모델은 물론 다중 입력 또는 다중 출력 등 다양한 구성을 할 수 있습니다.

---

### 케라스 주요 특징

케라스는 아래 4가지의 주요 특징을 가지고 있습니다.

* 모듈화 (Modularity)
    * 케라스에서 제공하는 모듈은 독립적이고 설정 가능하며, 가능한 최소한의 제약사항으로 서로 연결될 수 있습니다. 모델은 시퀀스 또는 그래프로 이러한 모듈들을 구성한 것입니다.
    * 특히 신경망 층, 비용함수, 최적화기, 초기화기법, 활성화함수, 정규화기법은 모두 독립적인 모듈이며, 새로운 모델을 만들기 위해 이러한 모듈을 조합할 수 있습니다.
* 최소주의 (Minimalism)
    * 각 모듈은 짥고 간결합니다.
    * 모든 코드는 한 번 훏어보는 것으로도 이해가능해야 합니다.
    * 단 반복 속도와 혁신성에는 다소 떨어질 수가 있습니다. 
* 쉬운 확장성
    * 새로운 클래스나 함수로 모듈을 아주 쉽게 추가할 수 있습니다. 
    * 따라서 고급 연구에 필요한 다양한 표현을 할 수 있습니다. 
* 파이썬 기반
    * Caffe 처럼 별도의 모델 설정 파일이 필요없으며 파이썬 코드로 모델들이 정의됩니다.
    
이 멋진 케라스를 개발하고 유지보수하고 있는 사람은 구글 엔지니어인 프랑소와 쏠레(François Chollet)입니다.

---

### 케라스 기본 개념

케라스의 가장 핵심적인 데이터 구조는 바로 `모델`입니다. 케라스에서 제공하는 시퀀스 모델로 원하는 레이어를 쉽게 순차적으로 쌓을 수 있습니다. 다중 출력이 필요하는 등 좀 더 복잡한 모델을 구성하려면 케라스 함수 API를 사용하면 됩니다. 케라스로 딥러닝 모델을 만들 때는 다음과 같은 순서로 작성합니다. 다른 딥러닝 라이브러리와 비슷한 순서이지만 훨씬 직관적이고 간결합니다. 

1. 데이터셋 생성하기
    * 원본 데이터를 불러오거나 시뮬레이션을 통해 데이터를 생성합니다.
    * 데이터로부터 훈련셋, 검증셋, 시험셋을 생성합니다.
    * 이 때 딥러닝 모델의 학습 및 평가를 할 수 있도록 포맷 변환을 합니다.
1. 모델 구성하기
    * 시퀀스 모델을 생성한 뒤 필요한 레이어를 추가하여 구성합니다.
    * 좀 더 복잡한 모델이 필요할 때는 케라스 함수 API를 사용합니다.
1. 모델 학습과정 설정하기
    * 학습하기 전에 학습에 대한 설정을 수행합니다.
    * 손실 함수 및 최적화 방법을 정의합니다.
    * 케라스에서는 compile() 함수를 사용합니다.
1. 모델 학습시키기
    * 훈련셋을 이용하여 구성한 모델로 학습시킵니다.
    * 케라스에서는 fit() 함수를 사용합니다.
1. 학습과정 살펴보기
    * 모델 학습 시 훈련셋, 검증셋의 손실 및 정확도를 측정합니다.
    * 반복횟수에 따른 손실 및 정확도 추이를 보면서 학습 상황을 판단합니다.
1. 모델 평가하기 
    * 준비된 시험셋으로 학습한 모델을 평가합니다.
    * 케라스에서는 evaluate() 함수를 사용합니다.
1. 모델 사용하기 
    * 임의의 입력으로 모델의 출력을 얻습니다.
    * 케라스에서는 predict() 함수를 사용합니다.

손글씨 영상을 분류하는 모델을 케라스로 간단히 구현해봤습니다. 가로세로 픽셀이 28 x 28인 이미지를 입력받아 이를 784 벡터로 구성한 다음 이를 학습 및 평가하는 코드입니다. 이 간단한 코드로 93.4%의 정확도 결과를 얻었습니다. 각 함수의 설명 및 인자에 대한 설명은 여러 모델을 실습해보면서 하나씩 설명드리겠습니다. 

In [2]:
# 0. 사용할 패키지 불러오기
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation

# 1. 데이터셋 생성하기
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 2. 모델 구성하기
model = Sequential()
model.add(Dense(units=64, input_dim=28*28, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

from keras import backend as K

interesting_class_id = 0

def single_class_accuracy(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    # Replace class_id_preds with class_id_true for recall here
    accuracy_mask = K.cast(K.equal(class_id_preds, interesting_class_id), 'int32')
    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    return class_acc

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy', single_class_accuracy])

# 4. 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=5, batch_size=32)

# 5. 학습과정 살펴보기
print('## training loss and acc ##')
print(hist.history['loss'])
print(hist.history['acc'])

# 6. 모델 평가하기
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

# 7. 모델 사용하기
xhat = x_test[0:1]
yhat = model.predict(xhat)
print('## yhat ##')
print(yhat)

Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.6672 - acc: 0.8283 - single_class_accuracy: 0.0000e+00     
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.3500 - acc: 0.9015 - single_class_accuracy: 0.0000e+00     
Epoch 3/5
15648/60000 [======>.......................] - ETA: 3s - loss: 0.3310 - acc: 0.9050 - single_class_accuracy: 0.0000e+00

KeyboardInterrupt: 


---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/lecture/)
* 다음 : [맥에서 케라스 설치하기](https://tykimos.github.io/2017/08/07/Keras_Install_on_Mac/)
* 다음 : [윈도우에서 케라스 설치하기](https://tykimos.github.io/2017/08/07/Keras_Install_on_Windows/)
* 다음 : [데이터셋 이야기](https://tykimos.github.io/2017/03/25/Dataset_and_Fit_Talk/)